# Getting Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/food_classifer_dataset.zip" -d "/content/"

Streaming output truncated to the last 5000 lines.
  inflating: /content/classifer_dataset/train/Softdrinks/softdrinks1722.jpg  
  inflating: /content/classifer_dataset/train/Softdrinks/softdrinks1723.jpg  
  inflating: /content/classifer_dataset/train/Softdrinks/softdrinks1724.jpg  
  inflating: /content/classifer_dataset/train/Softdrinks/softdrinks1725.jpg  
  inflating: /content/classifer_dataset/train/Softdrinks/softdrinks1726.jpg  
  inflating: /content/classifer_dataset/train/Softdrinks/softdrinks1727.jpg  
  inflating: /content/classifer_dataset/train/Softdrinks/softdrinks1728.jpg  
  inflating: /content/classifer_dataset/train/Softdrinks/softdrinks1729.jpg  
  inflating: /content/classifer_dataset/train/Softdrinks/softdrinks173.jpg  
  inflating: /content/classifer_dataset/train/Softdrinks/softdrinks1730.jpg  
  inflating: /content/classifer_dataset/train/Softdrinks/softdrinks1731.jpg  
  inflating: /content/classifer_dataset/train/Softdrinks/softdrinks1732.jpg  
  inflating: /

# Importing Important Libraries

In [ ]:
# this is going to help in data processing
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from keras.applications.inception_v3 import InceptionV3


# Setting up Variables

In [ ]:
# dimensions of our images.
img_width, img_height = 150, 150

#setting up the directories
train_data_dir = '/content/classifer_dataset/train'
validation_data_dir = '/content/classifer_dataset/validate'

#setting up the batchsizes.
nb_train_samples = 8400
nb_validation_samples = 1600
epochs = 50
batch_size = 16


# Building the Model

In [ ]:
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), # Shape of our images
                                include_top = False, # Leave out the last fully connected layers
                                weights = 'imagenet')

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
# freezing the pre trained layers

for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
from keras.layers import *
from keras import Model
from keras.optimizers import RMSprop


In [ ]:

# Flatten the output layer to 1 dimension
x = Flatten()(pre_trained_model.output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.2
x = BatchNormalization()(x)
x = Dropout(0.5)(x)          
# Add a final sigmoid layer for classification
x = Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.01), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

In [ ]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

# Data Augmentation

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1./ 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

#this generates batches of augment data for training
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 8400 images belonging to 2 classes.


In [ ]:
# this is the augmentation configuration we will use for validating
val_datagen = ImageDataGenerator(rescale=1./255)

#this generates batches of augment data for validating
validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 1600 images belonging to 2 classes.


#Training the model

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
525/525 [==============================] - 94s 180ms/step - loss: 0.1891 - acc: 0.9380 - val_loss: 0.2486 - val_acc: 0.9406
Epoch 2/50
525/525 [==============================] - 94s 178ms/step - loss: 0.1358 - acc: 0.9552 - val_loss: 0.2350 - val_acc: 0.9631
Epoch 3/50
525/525 [==============================] - 95s 180ms/step - loss: 0.1373 - acc: 0.9564 - val_loss: 0.1199 - val_acc: 0.9581
Epoch 4/50
525/525 [==============================] - 94s 179ms/step - loss: 0.1198 - acc: 0.9617 - val_loss: 0.0743 - val_acc: 0.9700
Epoch 5/50
525/525 [==============================] - 94s 179ms/step - loss: 0.1213 - acc: 0.9617 - val_loss: 0.0983 - val_acc: 0.9712
Epoch 6/50
525/525 [==============================] - 95s 181ms/step - loss: 0.1068 - acc: 0.9636 - val_loss: 0.1179 - val_acc: 0.9638
Epoch 7/50
525/525 [==============================] - 95s 181ms/step - loss: 0.1273 - acc: 0.9608 - val_loss: 0.14

# Saving Model

In [ ]:
#to save the weights in the model as a HDFS file

model.save_weights('/content/drive/My Drive/tr_model_weight.h5')


In [ ]:
#to save the architecture of the model as a json file

with open('/content/drive/My Drive/tr_model_architecture.json','w') as f:
    f.write(model.to_json())